# Install Package, Library, and Datasets

In [5]:
!nvidia-smi

Fri Feb  6 02:11:18 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 565.57.01              Driver Version: 565.57.01      CUDA Version: 12.7     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA A100-SXM4-80GB          On  |   00000000:B7:00.0 Off |                    0 |
| N/A   37C    P0             67W /  400W |       1MiB /  81920MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----

In [4]:
!pip install gdown nltk transformers transformers[torch] g2p_en datasets soundfile evaluate jiwer


[notice] A new release of pip is available: 24.2 -> 26.0.1
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
!gdown 1Uw4lAW28xDh5tijF4on32CtvxsatzEPi

Downloading...
From (original): https://drive.google.com/uc?id=1Uw4lAW28xDh5tijF4on32CtvxsatzEPi
From (redirected): https://drive.google.com/uc?id=1Uw4lAW28xDh5tijF4on32CtvxsatzEPi&confirm=t&uuid=6792cb7d-35aa-4bd3-aa4a-4ec5f3b79629
To: /workspace/LibriSpeech_final.zip
100%|██████████████████████████████████████| 2.45G/2.45G [00:35<00:00, 68.4MB/s]


In [11]:
!apt update -y && apt install unzip

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:2 http://archive.ubuntu.com/ubuntu jammy InRelease                         
Hit:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease                 
Hit:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease               
Hit:5 http://security.ubuntu.com/ubuntu jammy-security InRelease               
Hit:6 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease   
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
136 packages can be upgraded. Run 'apt list --upgradable' to see them.
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
unzip is already the newest version (6.0-26ubuntu3.2).
0 upgraded, 0 newly installed, 0 to remove and 136 not upgraded.


In [12]:
!apt install zip -y

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
zip is already the newest version (3.0-12build2).
0 upgraded, 0 newly installed, 0 to remove and 136 not upgraded.


In [2]:
!unzip -o LibriSpeech_final.zip

Archive:  LibriSpeech_final.zip
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0014.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0019.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0029.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0044.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0025.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/76263.trans.txt  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0026.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0023.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0043.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0030.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0032.wav  
  inflating: LibriSpeech_final/train-clean/4441/76263/4441-76263-0024.wav  
  inflating: LibriSpeech_final/train-clean/4441/76262/4441-7

# Import Library

In [109]:
import torch
import os
import soundfile as sf
from g2p_en import G2p
from datasets import Dataset
from transformers import (
    Wav2Vec2Processor,
    Wav2Vec2CTCTokenizer,
    Wav2Vec2ForCTC,
    TrainingArguments,
    Trainer,
    BatchEncoding
)
from dataclasses import dataclass
from typing import Dict, List, Union
from torch.nn.utils.rnn import pad_sequence
from torch import nn

import evaluate
import numpy as np
from collections import Counter
import json

import nltk
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

# Fine Tuning

## Change Device to Cuda

In [9]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using: {device}")

Using: cuda


## Load Pretrained Model

In [114]:
MODEL_ID = "moxeeeem/wav2vec2-finetuned-pronunciation-correction"

processor = Wav2Vec2Processor.from_pretrained(MODEL_ID)
model = Wav2Vec2ForCTC.from_pretrained(MODEL_ID).to(device)

Loading weights:   0%|          | 0/424 [00:00<?, ?it/s]

## Create New Vocab and New Tokenizer

In [115]:
ARPABET_TO_IPA = {
    # Vowels (Monophthongs)
    'AA': 'ɑ', 'AA0': 'ɑ', 'AA1': 'ɑ', 'AA2': 'ɑ',       # bot
    'AE': 'æ', 'AE0': 'æ', 'AE1': 'æ', 'AE2': 'æ',       # bat
    'AH': 'ʌ', 'AH0': 'ə', 'AH1': 'ʌ', 'AH2': 'ʌ',       # butt (stressed vs. unstressed)
    'AO': 'ɔ', 'AO0': 'ɔ', 'AO1': 'ɔ', 'AO2': 'ɔ',       # bought
    'EH': 'ɛ', 'EH0': 'ɛ', 'EH1': 'ɛ', 'EH2': 'ɛ',       # bet
    'ER': 'ɝ', 'ER0': 'ɚ', 'ER1': 'ɝ', 'ER2': 'ɝ',       # bird (stressed vs. unstressed)
    'IH': 'ɪ', 'IH0': 'ɪ', 'IH1': 'ɪ', 'IH2': 'ɪ',       # bit
    'IY': 'i', 'IY0': 'i', 'IY1': 'i', 'IY2': 'i',       # beat
    'UH': 'ʊ', 'UH0': 'ʊ', 'UH1': 'ʊ', 'UH2': 'ʊ',       # book
    'UW': 'u', 'UW0': 'u', 'UW1': 'u', 'UW2': 'u',       # boot

    # Vowels (Diphthongs)
    'AW': 'aʊ', 'AW0': 'aʊ', 'AW1': 'aʊ', 'AW2': 'aʊ',     # bout
    'AY': 'aɪ', 'AY0': 'aɪ', 'AY1': 'aɪ', 'AY2': 'aɪ',     # bite
    'EY': 'eɪ', 'EY0': 'eɪ', 'EY1': 'eɪ', 'EY2': 'eɪ',     # bait
    'OW': 'oʊ', 'OW0': 'oʊ', 'OW1': 'oʊ', 'OW2': 'oʊ',     # boat
    'OY': 'ɔɪ', 'OY0': 'ɔɪ', 'OY1': 'ɔɪ', 'OY2': 'ɔɪ',     # boy

    # Consonants
    'P': 'p', 'B': 'b', 'T': 't', 'D': 'd', 'K': 'k', 'G': 'g',
    'CH': 'tʃ', 'JH': 'dʒ', 'F': 'f', 'V': 'v', 'TH': 'θ', 'DH': 'ð',
    'S': 's', 'Z': 'z', 'SH': 'ʃ', 'ZH': 'ʒ', 'HH': 'h', 'M': 'm',
    'N': 'n', 'NG': 'ŋ', 'L': 'l', 'R': 'ɹ', 'W': 'w', 'Y': 'j',

    # Syllabic Consonants (less common, but useful for completeness)
    'EM': 'm̩', 'EN': 'n̩', 'EL': 'l̩', 'NX': 'ɾ̃',
}

SPECIAL_TOKENS = ["[PAD]", "[UNK]"]

In [116]:
def extract_unique_ipa_chars(arpabet_to_ipa):
    '''
    Exreact unique IPA characters
    '''
    vocab = set()
    for ipa in arpabet_to_ipa.values():
        for ch in ipa:
            vocab.add(ch)
    return sorted(vocab)

In [117]:
tokenizer = processor.tokenizer
vocab_dict = tokenizer.get_vocab()

old_vocab = list(vocab_dict.keys())

print(f"Vocab size: {len(old_vocab)}")
print(old_vocab)

Vocab size: 41
['[PAD]', '[UNK]', 'a', 'b', 'd', 'e', 'f', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 's', 't', 'u', 'v', 'w', 'z', '|', 'æ', 'ð', 'ŋ', 'ɑ', 'ɔ', 'ə', 'ɚ', 'ɛ', 'ɡ', 'ɪ', 'ɹ', 'ʃ', 'ʊ', 'ʌ', 'ʒ', '͡', 'θ', '<s>']


In [118]:
ipa_vocab = extract_unique_ipa_chars(ARPABET_TO_IPA)

vocab_list = SPECIAL_TOKENS + [c for c in ipa_vocab if c not in SPECIAL_TOKENS]
new_tokens = [t for t in vocab_list if t not in old_vocab]

tokenizer.add_tokens(new_tokens)

5

In [119]:
print(f"New token: {new_tokens}")

New token: ['g', 'ɝ', 'ɾ', '̃', '̩']


In [120]:
num_new_tokens = len(tokenizer)
print(num_new_tokens)

45


In [121]:
model.config.vocab_size = len(tokenizer)

In [122]:
old_lm_head = model.lm_head
old_vocab_size = old_lm_head.out_features
hidden_size = old_lm_head.in_features

if num_new_tokens > old_vocab_size:
    new_lm_head = nn.Linear(hidden_size, num_new_tokens)
    
    new_lm_head.weight.data[:old_vocab_size, :] = old_lm_head.weight.data
    new_lm_head.bias.data[:old_vocab_size] = old_lm_head.bias.data
    
    model.lm_head = new_lm_head

In [134]:
processor.tokenizer = tokenizer

## Utilities Function for Preprocessing Data

In [135]:
g2p = G2p()

def phonemize_text(text):
    """
    Convert text → ARPAbet → IPA
    Args:
        text (str): text to convert into phonemes

    Returns:
        list: Contains of list chars of ipa phonemes
    """
    arpabet_phonemes = [p for p in g2p(text) if p.strip() and p not in [' ', ',', '.', '!', '?']]
    
    ipa = []
    for p in arpabet_phonemes:
        ipa_symbol = ARPABET_TO_IPA.get(p)
        if ipa_symbol:
            ipa.append(ipa_symbol)
    return ipa

def preprocess_dataset(dataset):
    """Add 'phonemes' column mapping"""
    dataset = dataset.map(lambda batch: {"phonemes": phonemize_text(batch["text"])})
    return dataset

In [136]:
def load_librispeech_folder(base_dir):
    """
    Load all .wav files and corresponding transcripts from a Librispeech-style folder.
    
    Args:
        base_dir (str): Path to train-clean or test-clean folder.
        
    Returns:
        datasets.Dataset: Contains 'audio' (path) and 'text' (transcript) columns.
    """
    samples = []
    
    for speaker in os.listdir(base_dir):
        spk_path = os.path.join(base_dir, speaker)
        if not os.path.isdir(spk_path):
            continue
        
        for chapter in os.listdir(spk_path):
            chap_path = os.path.join(spk_path, chapter)
            if not os.path.isdir(chap_path):
                continue
            
            trans_file = [f for f in os.listdir(chap_path) if f.endswith(".trans.txt")]
            if len(trans_file) != 1:
                continue
            trans_path = os.path.join(chap_path, trans_file[0])
            
            with open(trans_path, "r") as f:
                lines = f.readlines()
            
            trans_dict = {}
            for line in lines:
                parts = line.strip().split(" ", 1)
                if len(parts) == 2:
                    file_id, text = parts
                    trans_dict[file_id] = text
            
            for file in os.listdir(chap_path):
                if file.endswith(".wav"):
                    file_id = os.path.splitext(file)[0]
                    if file_id in trans_dict:
                        samples.append({
                            "audio": os.path.join(chap_path, file),
                            "text": trans_dict[file_id]
                        })
    
    if len(samples) == 0:
        raise ValueError(f"No valid samples found in {base_dir}")
    
    return Dataset.from_list(samples)

In [137]:
def prepare_batch(batch):
    """
    Preprocessing each batch data for fine tuning
    """
    speech, sr = sf.read(batch["audio"])
    inputs = processor(speech, sampling_rate=sr, return_attention_mask=True)

    labels = processor.tokenizer(batch["phonemes"]).input_ids

    batch["input_values"] = inputs.input_values[0]
    batch["attention_mask"] = inputs.attention_mask[0]
    batch["labels"] = labels
    return batch

## Load and Preprocessing Datasets

In [138]:
train_ds = load_librispeech_folder("LibriSpeech_final/train-clean")
test_ds  = load_librispeech_folder("LibriSpeech_final/test-clean")

train_ds = preprocess_dataset(train_ds)
test_ds  = preprocess_dataset(test_ds)

train_ds = train_ds.map(
    prepare_batch,
    remove_columns=train_ds.column_names,
    num_proc=4
)

test_ds = test_ds.map(
    prepare_batch,
    remove_columns=test_ds.column_names,
    num_proc=4
)

Map:   0%|          | 0/7051 [00:00<?, ? examples/s]

Map:   0%|          | 0/453 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/7051 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/453 [00:00<?, ? examples/s]

In [139]:
print(train_ds[0])
print(len(train_ds))
print(train_ds.column_names)

print(test_ds[0])
print(len(test_ds))

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



## Finetune Pretrained Model

In [140]:
@dataclass
class DataCollatorCTC:
    """
    Data collator for Wav2Vec2 Connectionist Temporal Classification (CTC) training.

    This collator is used with Hugging Face Trainer to prepare batches of audio features 
    and corresponding tokenized labels for training a Wav2Vec2 model with CTC loss.

    Key responsibilities:
    1. Pads input audio sequences to the same length using the processor's feature extractor.
    2. Pads tokenized labels to the same length using the tokenizer.
       - Labels are wrapped in BatchEncoding objects to ensure proper padding.
    3. Replaces pad token IDs in the labels with -100 to tell PyTorch's CTC loss
       to ignore these positions.

    Attributes:
        processor (Wav2Vec2Processor): The processor wrapping the feature extractor and tokenizer.
        padding (bool | str): Padding strategy passed to `processor.pad()` and tokenizer.pad().
            - True: pad to the longest sequence in the batch
            - 'longest': same as True
            - 'max_length': pad to the model's maximum input length
    """
    def __init__(self, processor, padding=True):
        self.processor = processor
        self.padding = padding

    def __call__(self, features: List[Dict]) -> Dict[str, torch.Tensor]:
        """
        Collate a batch of audio and label features.

        Args:
            features (List[Dict]): List of dataset items, each a dictionary containing:
                - "input_values": audio features (list or tensor of floats)
                - "labels": tokenized label sequence (list of ints)

        Returns:
            Dict[str, torch.Tensor]: A batch dictionary with:
                - "input_values": padded audio features (torch.Tensor)
                - "attention_mask": attention masks corresponding to audio features
                - "labels": padded labels with pad token IDs replaced by -100
        """
        input_features = [{"input_values": f["input_values"]} for f in features]
        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            return_tensors="pt"
        )

        labels = []
        for f in features:
            if any(isinstance(i, list) for i in f["labels"]):
                flat_labels = [i for sub in f["labels"] for i in sub]
            else:
                flat_labels = f["labels"]
            labels.append(flat_labels)

        labels_batch = self.processor.tokenizer.pad(
            {"input_ids": labels},
            padding=True,
            return_tensors="pt"
        )

        batch["labels"] = labels_batch["input_ids"]
        batch["labels"][batch["labels"] == self.processor.tokenizer.pad_token_id] = -100

        return batch

In [141]:
cer_metric = evaluate.load("cer")

def compute_metrics(pred):
    """
    Compute CER (Character Error Rate) for Wav2Vec2 CTC.
    pred: PredictionOutput dari Trainer
    """
    pred_logits = pred.predictions
    pred_ids = np.argmax(pred_logits, axis=-1)

    pred_str = processor.batch_decode(pred_ids)  # ignore -100
    label_ids = pred.label_ids
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    label_str = processor.batch_decode(label_ids, group_tokens=False)

    cer = cer_metric.compute(predictions=pred_str, references=label_str)

    return {"cer": cer}

In [145]:
data_collator = DataCollatorCTC(processor)

training_args = TrainingArguments(
    output_dir="./lingulu_wav2vec2_pronunciation_finetune_checkpoint",
    group_by_length=False,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=8,
    eval_strategy="steps",
    num_train_epochs=10,
    save_steps=50,
    eval_steps=50,
    save_total_limit=3,
    learning_rate=1e-5,
    logging_steps=50,
    gradient_accumulation_steps=2,
    bf16=True,
    fp16=False,
    load_best_model_at_end=True,
    resume_from_checkpoint=True,
    remove_unused_columns=False
)

In [146]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=test_ds,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

trainer.train()

Step,Training Loss,Validation Loss,Cer
50,3.070491,0.521950,0.120490
100,0.953462,0.334930,0.086068
150,0.775154,0.284527,0.073130
200,0.701032,0.259461,0.068415
250,0.626419,0.244569,0.066218
300,0.588775,0.232616,0.064745
350,0.582970,0.223401,0.063111
400,0.570465,0.214302,0.061075
450,0.524862,0.207210,0.058075
500,0.507323,0.201749,0.056495


Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

...

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

TrainOutput(global_step=2210, training_loss=0.5271613395052258, metrics={'train_runtime': 15134.284, 'train_samples_per_second': 4.659, 'train_steps_per_second': 0.146, 'total_flos': 3.4656251748351803e+19, 'train_loss': 0.5271613395052258, 'epoch': 10.0})

## Export Train History

In [147]:
import pandas as pd
pd.DataFrame(trainer.state.log_history).to_csv("train_history.csv", index=False)

## Export Model and Processor

In [148]:
model.save_pretrained("./lingulu_wav2vec2_pronounciation_finetune")
processor.save_pretrained("./lingulu_wav2vec2_pronounciation_finetune")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

['./lingulu_wav2vec2_pronounciation_finetune/processor_config.json']

In [ ]:
!zip -r lingulu_wav2vec2_pronounciation_finetune.zip lingulu_wav2vec2_pronounciation_finetune/

  adding: lingulu_wav2vec2_pronounciation_finetune/ (stored 0%)
  adding: lingulu_wav2vec2_pronounciation_finetune/processor_config.json (deflated 43%)
  adding: lingulu_wav2vec2_pronounciation_finetune/vocab.json (deflated 51%)
  adding: lingulu_wav2vec2_pronounciation_finetune/added_tokens.json (deflated 34%)
  adding: lingulu_wav2vec2_pronounciation_finetune/tokenizer_config.json (deflated 78%)
  adding: lingulu_wav2vec2_pronounciation_finetune/model.safetensors